# NF264- Project 2: Digit recognizer
TODO: Write this better

We  are  working  for  a  small  company  that  provides  machinelearning  solutions  for  its  customers.   The  postal  office  needs an  AI system  to  automatically  deliver  mail.   As  a  part  of  the  system,  they  need a  computer  program  that  recognises  handwritten  digits. We are providing  this  program  and  as machine  learning  experts, we write  the code that produces a classifier and this report that describes what we have done.

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import tensorflow as tf 
from sklearnex import patch_sklearn
patch_sklearn()

C:\Users\elias\Miniconda3\envs\tf\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning:

Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.

C:\Users\elias\Miniconda3\envs\tf\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning:

Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.

C:\Users\elias\Miniconda3\envs\tf\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning:

Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.

C:\Users\elias\Miniconda3\envs\tf\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning:

Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be unders

## The Dataset
TODO: Comment what the dataset is

28x28 pixel images with black/white value 0-255
How many rows
How are they given? list of list, each image is 784 ints

In [2]:
X = pd.read_csv('handwritten_digits_images.csv', header=None).to_numpy()
y = pd.read_csv('handwritten_digits_labels.csv', header=None).to_numpy()
print(X.shape)

(70000, 784)


## Preprocessing steps
TODO: Comment what we do before splitting and traiing and why
Make labels categorical
Normalize values to 0-1

In [3]:
from keras.utils import to_categorical

y = to_categorical(y)

# Normalize to range 0-1
X = X.astype('float32')
X = X / 255.0

Using TensorFlow backend.


In [4]:
print(y[15000])

px.imshow(X[15000].reshape(28,28), color_continuous_scale=["white", "black"])

[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]


## Splitting data
TODO: comment how we split 70 15 15

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_val_test, y_train, y_val_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_val_test, y_val_test, test_size=0.5, random_state=42)

print('Train: X=%s, y=%s' % (X_train.shape, y_train.shape))
print('Test: X=%s, y=%s' % (X_test.shape, y_test.shape))
print('Validate: X=%s, y=%s' % (X_val.shape, y_val.shape))

Train: X=(49000, 784), y=(49000, 10)
Test: X=(10500, 784), y=(10500, 10)
Validate: X=(10500, 784), y=(10500, 10)


## Candidate  algorithms  and  choice  of  candidate  hyperparameters  (and why were the others left out)


Firstly we want to chose a classifier algorithm since a classifier utilizes some training data to understand how given input variables relate to a class. In this case, pictures of integers 0-9 are used as the training data. When the classifier is trained accurately, it can be used to detect integers for the Postal office. 

There are many candidates in this space. Firstly, we choose K Nearest Neighbors Classifier since this is kind of a baseline model, and we have implemnted this model from scratch is previous courses so we are well aware of the algorithm.

Secondly, we want to test a descision tree classifier since we know that this is an effective Classifier from our previous Project, where we classified a dataset with 10 features. When we think about a writtin digit, there are probably some descisions thart could be made in a descision tree, such as if it has a single line in vertical direction it is a 1 or 7, or if it contains two circles it is a 8. From the pixel data we ecpect there will be some kind of denominator that could classify the image into a category of digits.

Lastly, we  want to explore a Sequential Convolutional Neural Network Classifier since we are not as familier with this tool, and want to learn more about implemtning this Classifier. Neural Nets can be very powerfull if trained accuratly, so we want to explore if this could be a feasible solution for recognizing digits.

## Chosen performance measure
TODO: Write a bit more

When chosing perfomance measure there are several that could be used, i.e MSE and RMSE, but we want to use the accuracy in percentage (0-100%) on the test data for model selection, and the accuracy on validation data for model evaluation.

## K Nearest Neighbors Classifier
The K-nearest neighbors (KNN) algorithm is a data classification method for estimating the probability that a data point will become a member of one or another group based on which group the data points are closest to it. A classification problem has a discrete value as its output. It is a type of supervised machine learning algorithm used to solve classification (and regression) problems. The algorithm is also called a lazy learning and non-parametic algorithm. This is because it is lazy and dosen’t preform any training when you supply the training data. It just stores the data during the training time and does not perform any calculations. The KNN algorithm does not build a model until a query is performed on the data set. It is also considered a non-parametric methods because it does not make any assumptions about the underlying data distribution. KNN tries to determine what group a data point belongs to by looking at the data points around it. It also involves classifying a data point by looking at the nearest annotated data point.

KNN is a supervised classification algorithm that classifies new data points based on the nearest data points. A advantage of using it, is that the training phase of K-nearest neighbor classification is much faster compared to other classification algorithms. There is no need to train a model for generalization, that is why KNN is known as the simple and instance-based learning algorithm. One disadvantage of using KNN is that the testing phase of K-nearest neighbor classification is slower and costlier in terms of time and memory. It requires large memory for storing the entire training dataset for prediction.

### Hyperparameters
TODO: What parameters are there?
k

In [6]:
from sklearn.neighbors import KNeighborsClassifier

kVals = [1, 2, 3, 4, 5, 6, 7, 8 , 9, 10, 15, 30]
accuracies = []

for k in kVals: # Testing many k hyperparametyers to optimize performance
	model = KNeighborsClassifier(algorithm='auto', n_neighbors=k)
	model.fit(X_train, y_train)

	score = model.score(X_val, y_val)
	print("k=%d, validation accuracy=%.2f%%" % (k, score * 100))
	accuracies.append([k, score * 100])

k=1, validation accuracy=96.73%
k=2, validation accuracy=94.48%
k=3, validation accuracy=96.65%
k=4, validation accuracy=95.26%
k=5, validation accuracy=96.28%
k=6, validation accuracy=95.30%
k=7, validation accuracy=96.04%
k=8, validation accuracy=95.34%
k=9, validation accuracy=95.82%
k=10, validation accuracy=94.96%
k=15, validation accuracy=95.21%
k=30, validation accuracy=93.80%


In [7]:
#Plotting data
df = pd.DataFrame(accuracies, columns = ['k', 'Accuracy'])
px.line(df, x="k", y = 'Accuracy', title="kNN Model accuracy on validation data")

In [10]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

model = KNeighborsClassifier(n_neighbors=1)
model.fit(X_train, y_train)
predictions = model.predict(X_test)

# show a final classification report demonstrating the accuracy of the classifier
# for each of the digits
print("EVALUATION ON TESTING DATA")
print(classification_report(y_test, predictions))


print("sklearn KNeighborsClassifier Test data accuracy: {:3.2f}%".format(accuracy_score(y_test, predictions)*100))

EVALUATION ON TESTING DATA
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       972
           1       0.97      0.99      0.98      1177
           2       0.98      0.98      0.98      1008
           3       0.96      0.96      0.96      1031
           4       0.98      0.98      0.98      1071
           5       0.96      0.96      0.96      1026
           6       0.98      0.98      0.98      1063
           7       0.97      0.97      0.97      1085
           8       0.98      0.93      0.96      1038
           9       0.95      0.96      0.95      1029

   micro avg       0.97      0.97      0.97     10500
   macro avg       0.97      0.97      0.97     10500
weighted avg       0.97      0.97      0.97     10500
 samples avg       0.97      0.97      0.97     10500

sklearn KNeighborsClassifier Test data accuracy: 97.10%


from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier


estimator_KNN = KNeighborsClassifier(algorithm='auto')
parameters_KNN = {
    'n_neighbors': (1,2,3,4,5,10),
    'leaf_size': (20,40,1),
    'p': (1,2),
    'weights': ('uniform', 'distance'),
    'metric': ('minkowski', 'chebyshev')}
                   
# with GridSearch
grid_search_KNN = GridSearchCV(
    estimator=estimator_KNN,
    param_grid=parameters_KNN,
    scoring = 'accuracy',
    n_jobs = -1,
    cv = 5,
    verbose=1
)

KNN=grid_search_KNN.fit(X_train, y_train)

print(grid_search_KNN.best_params_ ) 
#Mean cross-validated score of the best_estimator
print('Best Score - KNN:', grid_search_KNN.best_score_ )


### kNN Findings
TODO: Results from evaluation

## Decision Tree Classifier
Decision Tree is a Supervised Machine Learning Algorithm that uses a set of rules to make decisions. A decision tree has a flowchart-like tree structure where an internal node represents feature, the branch represents a decision rule, and each leaf node represents the outcome. 

The most important feature is the capability of capturing descriptive decisionmaking knowledge from the supplied data. Decision tree can be generated from training sets. A decision tree classifier generates the actual prediction at the leaf nodes, more information can be stored at the leaf nodes. The decision tree is a distribution-free or non-parametric method, which does not depend upon probability distribution assumptions. It can handle high dimensional data with good accuracy. 


### Hyperparameters
TODO: What parameters are there?

In [11]:
from sklearn.tree import DecisionTreeClassifier

criterion = ['gini','entropy']
max_depth = [3,4,5,6,7,8,9,10]

df = []
for d in max_depth:
    for c in criterion:
        dtc = DecisionTreeClassifier(criterion=c, max_depth=d)
        dtc.fit(X_train, y_train)
        acc = accuracy_score(y_val, dtc.predict(X_val))*100
        df.append([d,c,acc])
        print("Descicion tree with max depth: "+str(d) + ", impurity measure: "+c+", Accuracy: {:3.2f}%".format(acc))

Descicion tree with max depth: 3, impurity measure: gini, Accuracy: 24.28%
Descicion tree with max depth: 3, impurity measure: entropy, Accuracy: 30.12%
Descicion tree with max depth: 4, impurity measure: gini, Accuracy: 39.70%
Descicion tree with max depth: 4, impurity measure: entropy, Accuracy: 46.63%
Descicion tree with max depth: 5, impurity measure: gini, Accuracy: 57.43%
Descicion tree with max depth: 5, impurity measure: entropy, Accuracy: 58.47%
Descicion tree with max depth: 6, impurity measure: gini, Accuracy: 65.04%
Descicion tree with max depth: 6, impurity measure: entropy, Accuracy: 68.11%
Descicion tree with max depth: 7, impurity measure: gini, Accuracy: 72.08%
Descicion tree with max depth: 7, impurity measure: entropy, Accuracy: 74.56%
Descicion tree with max depth: 8, impurity measure: gini, Accuracy: 75.38%
Descicion tree with max depth: 8, impurity measure: entropy, Accuracy: 78.70%
Descicion tree with max depth: 9, impurity measure: gini, Accuracy: 81.53%
Descici

In [12]:
#Plotting data
df = pd.DataFrame(df, columns = ['Depth', 'Impurity', 'Accuracy'])
px.line(df, x="Depth", y = 'Accuracy', color='Impurity', title="DecisionTreeClassifier accuracy on validation data")

## Sequential Convolutional Neural Network Classifier
TODO: What is Sequential Convolutional Neural Network Classifier?

https://en.wikipedia.org/wiki/Convolutional_neural_network

### Hyperparameters

Layers
Activation function
keras.optimizers import SGD

In [4]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.optimizers import Adam
from sklearn.model_selection import KFold
from keras.layers import BatchNormalization

print(tf.__version__)

if tf.test.gpu_device_name(): 
    print('GPU Device:{}'.format(tf.test.gpu_device_name()))

1.10.0
GPU Device:/device:GPU:0


In [10]:
X = X.reshape(X.shape[0], 28, 28, 1)
X_train, X_val_test, y_train, y_val_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_val_test, y_val_test, test_size=0.5, random_state=42)

In [ ]:
def neural_net_1(X_train, y_train, X_val, y_val, X_test, y_test, epochs = 5):
	model = Sequential()
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(28, 28, 1)))
	model.add(MaxPooling2D((2, 2)))
	model.add(Flatten())
	model.add(Dense(100, activation='relu', kernel_initializer='he_uniform'))
	opt = Adam(lr=0.1)
	model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

	model.fit(X_train, y_train, epochs=epochs, batch_size=32, validation_data=(X_val, y_val), verbose=1)
	_, acc = model.evaluate(X_test, y_test, verbose=1)
	print('Model 1 accuracy on test data:  %.3f' % (acc * 100.0))

	return model

neural_net = neural_net_1(X_train, y_train, X_val, y_val, X_test, y_test, epochs=3)

In [12]:
def neural_net(X_train, y_train, X_val, y_val, X_test, y_test, epochs = 5):
	model = Sequential()
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(28, 28, 1)))
	model.add(MaxPooling2D((2, 2)))
	model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform'))
	model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform'))
	model.add(MaxPooling2D((2, 2)))
	model.add(Flatten())
	model.add(Dense(100, activation='relu', kernel_initializer='he_uniform'))
	model.add(Dense(10, activation='softmax'))
	opt = SGD(lr=0.01, momentum=0.9)
	model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

	model.fit(X_train, y_train, epochs=epochs, batch_size=32, validation_data=(X_val, y_val), verbose=1)
	_, acc = model.evaluate(X_test, y_test, verbose=1)
	print('Model accuracy on test data:  %.3f' % (acc * 100.0))

	return model


neural_net = neural_net(X_train, y_train, X_val, y_val, X_test, y_test, epochs=5)

Train on 49000 samples, validate on 10500 samples
Epoch 1/5
49000/49000 [==============================] - 26s 533us/step - loss: 0.1378 - acc: 0.9577 - val_loss: 0.0801 - val_acc: 0.9754
Epoch 2/5
49000/49000 [==============================] - 24s 480us/step - loss: 0.0466 - acc: 0.9854 - val_loss: 0.0452 - val_acc: 0.9860
Epoch 3/5
49000/49000 [==============================] - 24s 497us/step - loss: 0.0310 - acc: 0.9909 - val_loss: 0.0454 - val_acc: 0.9858
Epoch 4/5
49000/49000 [==============================] - 26s 528us/step - loss: 0.0214 - acc: 0.9934 - val_loss: 0.0372 - val_acc: 0.9893
Epoch 5/5
10500/10500 [==============================] - 2s 210us/step
Model accuracy on test data:  98.571


Batch normalization can be used after convolutional and fully connected layers. It has the effect of changing the distribution of the output of the layer, specifically by standardizing the outputs. This has the effect of stabilizing and accelerating the learning process.

In [13]:
def neural_net(X_train, y_train, X_val, y_val, X_test, y_test, epochs = 5):
	model = Sequential()
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(28, 28, 1)))
	model.add(BatchNormalization())
	model.add(MaxPooling2D((2, 2)))
	model.add(Flatten())
	model.add(Dense(100, activation='relu', kernel_initializer='he_uniform'))
	model.add(BatchNormalization())
	model.add(Dense(10, activation='softmax'))
	opt = SGD(lr=0.01, momentum=0.9)
	model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

	model.fit(X_train, y_train, epochs=epochs, batch_size=32, validation_data=(X_val, y_val), verbose=1)
	_, acc = model.evaluate(X_test, y_test, verbose=1)
	print('Model accuracy on test data:  %.3f' % (acc * 100.0))

	return model


neural_net = neural_net(X_train, y_train, X_val, y_val, X_test, y_test, epochs=5)

Train on 49000 samples, validate on 10500 samples
Epoch 1/5
49000/49000 [==============================] - 27s 557us/step - loss: 0.1318 - acc: 0.9606 - val_loss: 0.0816 - val_acc: 0.9726
Epoch 2/5
49000/49000 [==============================] - 26s 526us/step - loss: 0.0496 - acc: 0.9851 - val_loss: 0.0627 - val_acc: 0.9812
Epoch 3/5
49000/49000 [==============================] - 26s 535us/step - loss: 0.0326 - acc: 0.9899 - val_loss: 0.0463 - val_acc: 0.9857
Epoch 4/5
49000/49000 [==============================] - 28s 571us/step - loss: 0.0202 - acc: 0.9943 - val_loss: 0.0473 - val_acc: 0.9849
Epoch 5/5
10500/10500 [==============================] - 2s 189us/step
Model accuracy on test data:  98.714


In [ ]:
def neural_net(X, y, epochs = 5, folds=3):
    scores = []
    hist = []
    kfold = KFold(folds, shuffle=True, random_state=1)
    
    for train_ix, test_ix in kfold.split(X):
        model = Sequential()
        model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(28, 28, 1)))
        model.add(BatchNormalization())
        model.add(MaxPooling2D((2, 2)))
        model.add(Flatten())
        model.add(Dense(100, activation='relu', kernel_initializer='he_uniform'))
        model.add(BatchNormalization())
        model.add(Dense(10, activation='softmax'))
        opt = SGD(lr=0.01, momentum=0.9)
        model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
        
        trainX, trainY, testX, testY = X[train_ix], y[train_ix], X[test_ix], y[test_ix]
        h = model.fit(trainX, trainY, epochs=epochs, batch_size=32, validation_data=(testX, testY), verbose=1)
        
        _, acc = model.evaluate(testX, testY, verbose=1)
        print('> %.3f' % (acc * 100.0))
        
        scores.append(acc)
        hist.append(h)
        
    return scores, hist
    

final = neural_net(X,y, epochs=10, folds=3)

In [ ]:
print(classification_report(y_test, final.predict(X_test)))

print("sklearn KNeighborsClassifier Test data accuracy: {:3.2f}%".format(accuracy_score(y_test, predictions)*100))

## Model selection schemes that you used.  Justify your choices

## What is your final classifier and how does it work.
Neural net
## How  well  it  is  expected  to  perform  in  production  (on  unseen  data).Justify your estimate

## Measures taken to avoid overfitting
Train/Validation and Test Split
Cross Validation

## Given more resources (time or computing resources),  how would youimprove your solution
More Epochs and K-folds if i had a stronger GPU. Also i had to use older versions of CUDA, CUDNN, Tenserflow and Keras to work with my GPU. Current versions of the library may be better optimized and produce better results. 

K-folds should be used to avoid overfitting to the specific data split, but i did not manage to use my GPU with CUDA for Tensorflow so the fit of the model took a lot of time. I would run the same function with 5 folds of splitting data.